<a href="https://colab.research.google.com/github/vedanshipathak/Mental-Health-Support-Chatbot/blob/main/Final_Model_mental_health_support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain chromadb openai flask datasets evaluate torch transformers accelerate faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s e

In [6]:
import pandas as pd
import re
# Load the dataset into a pandas DataFrame
df = pd.read_csv(r'/content/counselchat-data.csv')

# Function to strip HTML tags (like <p>, <br>, etc.)
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', str(text))

# Apply cleaning to the 'answerText' column
df['clean_answer'] = df['answerText'].apply(remove_html_tags)

# Optional: Combine questionTitle and questionText for better context
df['full_question'] = df['questionTitle'].fillna('') + ' ' + df['questionText'].fillna('')

# Preview the cleaned data
df[['full_question', 'clean_answer']].head()
# Display first few rows
print(df.head())


                 questionID                                     questionTitle  \
0  5566fab2a64752d71ec3ca69  Escalating disagreements between mother and wife   
1  5566f94fa64752d71ec3ca64          I'm addicted to smoking. How can I stop?   
2  5567d26887a1cc0c3f3d8f46                    Keeping secrets from my family   
3  556bed15c969ba5861709df5         The Underlying Causes of Being Possessive   
4  556ba115c969ba5861709de6         Can I control anxiety without medication?   

                                        questionText  \
0  My wife and mother are having tense disagreeme...   
1  I'm planning to have baby, so I have to quit s...   
2  I have secrets in my mind, and I don't know wh...   
3  I am extremely possessive in my relationships ...   
4  I had a head injury a few years ago and my min...   

                                         questionUrl  \
0  https://counselchat.com/questions/escalating-d...   
1  https://counselchat.com/questions/i-m-addicted...   
2  https

In [3]:
# Display column names
print("Column Names:", df.columns.tolist())


Column Names: ['questionID', 'questionTitle', 'questionText', 'questionUrl', 'topics', 'therapistName', 'therapistUrl', 'answerText', 'upvotes']


Sample Question: My wife and mother are having tense disagreements. In the past, they’ve had minor differences. For example, my wife would complain to me my mother is too overbearing; my mother would complain my wife is lazy.
However, it’s intensified lately. I think the cause is my wife talked back to her once. Now, any little disagreement is magnified, leading to major disagreements. What can I do?
Sample Answer: What you are describing is something psychologists have termed &#34;triangulation&#34; which is what happens when one family member will not talk to the one they have a problem with and goes to a third member of the family to complain instead. You have been &#34;triangulated&#34; by your wife and mother.This is often seen in families. It's seen everywhere. How many times have you had a problem with someone but you didn't go to them to tell them, you went to someone else to complain? It is usually difficult for a person to confront another, especially in relationships where t

In [9]:
import html

# Apply to the cleaned answer column
df['decoded_answer'] = df['clean_answer'].apply(html.unescape)

# Optional: also decode the question column if needed
df['decoded_question'] = df['full_question'].apply(html.unescape)

# Preview
df[['decoded_question', 'decoded_answer']].head()

,decoded_question,decoded_answer
0,Escalating disagreements between mother and wi...,What you are describing is something psycholog...
1,I'm addicted to smoking. How can I stop? I'm p...,Hi. Good for you in planning ahead to do what'...
2,Keeping secrets from my family I have secrets ...,It sounds like keeping the secrets has become ...
3,The Underlying Causes of Being Possessive I am...,Hi there. It's great you are able to realize t...
4,Can I control anxiety without medication? I ha...,You didn't say what or how many medications yo...


) missing from font(s) DejaVu Sans.
  plt.savefig(


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('decoded_question').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('decoded_answer').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

) missing from font(s) DejaVu Sans.
  fig.canvas.draw()
) missing from font(s) DejaVu Sans.
  plt.savefig(


from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['decoded_answer'].value_counts()
    for x_label, grp in _df_2.groupby('decoded_question')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('decoded_question')
_ = plt.ylabel('decoded_answer')

In [10]:
# Extract relevant columns
questions = df["decoded_question"].dropna().tolist()  # Remove NaN values if any
answers = df["decoded_answer"].dropna().tolist()

# Print a sample
print("Sample Question:", questions[0])
print("Sample Answer:", answers[0])

Sample Question: Escalating disagreements between mother and wife My wife and mother are having tense disagreements. In the past, they’ve had minor differences. For example, my wife would complain to me my mother is too overbearing; my mother would complain my wife is lazy.
However, it’s intensified lately. I think the cause is my wife talked back to her once. Now, any little disagreement is magnified, leading to major disagreements. What can I do?
Sample Answer: What you are describing is something psychologists have termed "triangulation" which is what happens when one family member will not talk to the one they have a problem with and goes to a third member of the family to complain instead. You have been "triangulated" by your wife and mother.This is often seen in families. It's seen everywhere. How many times have you had a problem with someone but you didn't go to them to tell them, you went to someone else to complain? It is usually difficult for a person to confront another, es

In [11]:
!pip install langchain openai


In [12]:
from langchain.prompts import PromptTemplate

# General Mental Health Support Prompt
general_prompt = PromptTemplate(
    template="""
You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: {client_message}
Therapist:
""",
    input_variables=["client_message"],
)

# Example usage:
client_message = "I've been feeling very anxious lately and don't know how to cope."
formatted_prompt = general_prompt.format(client_message=client_message)
print(formatted_prompt)



You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: I've been feeling very anxious lately and don't know how to cope.
Therapist:



In [13]:
def get_prompt(client_message):
    if any(word in client_message.lower() for word in ["anxious", "panic", "overwhelmed"]):
        return anxiety_prompt.format(client_message=client_message)
    elif any(word in client_message.lower() for word in ["depressed", "hopeless", "sad"]):
        return depression_prompt.format(client_message=client_message)
    elif any(word in client_message.lower() for word in ["relationship", "partner", "marriage", "family"]):
        return relationship_prompt.format(client_message=client_message)
    else:
        return general_prompt.format(client_message=client_message)

# Example Test
test_message = "I'm having serious conflicts with my wife, and I feel stuck."
selected_prompt = get_prompt(test_message)
print(selected_prompt)



You are a compassionate therapist. Respond with empathy, support, and actionable advice.
Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

Client: I'm having serious conflicts with my wife, and I feel stuck.
Therapist:



In [14]:
!pip install transformers torch accelerate


In [15]:
from huggingface_hub import login
login()


In [16]:
import requests

try:
    response = requests.get("https://huggingface.co", timeout=5)
    if response.status_code == 200:
        print("✅ Internet is working. Hugging Face is reachable!")
    else:
        print("⚠️ Internet issue: Received status code", response.status_code)
except requests.ConnectionError:
    print("❌ No internet connection detected!")


✅ Internet is working. Hugging Face is reachable!


In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load Mistral-7B model & tokenizer
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

print("Model loaded successfully!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Model loaded successfully!


In [18]:
def generate_response(client_message):
    # Format the structured prompt
    prompt = f"""
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Client: {client_message}
    Therapist:
    """

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=200, temperature=0.7, top_p=0.9)

    # Decode and return the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# ✅ Test the chatbot
test_message = "I've been feeling very anxious lately and don't know how to cope."
response = generate_response(test_message)
print("\nChatbot Response:\n", response)


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Chatbot Response:
 
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Client: I've been feeling very anxious lately and don't know how to cope.
    Therapist:
    
    I'm sorry to hear that you're feeling anxious. It's completely normal to experience anxiety at times, but it's important to address it when it becomes overwhelming.
    
    One helpful technique to manage anxiety is mindfulness meditation. This involves focusing on the present moment and accepting your thoughts and feelings without judgment. You can start by finding a quiet place to sit or lie down, closing your eyes, and focusing on your breath. As you breathe in and out, try to let go of any distracting thoughts and just be present in the moment.
    


In [19]:
# ✅ Test the chatbot
test_message = "I feel extremely depressed."
response = generate_response(test_message)
print("\nChatbot Response:\n", response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Chatbot Response:
 
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Client: I feel extremely depressed.
    Therapist:
    
    I'm sorry to hear that you're feeling depressed. It's important to remember that depression is a common and treatable mental health condition. It's okay to feel overwhelmed and unsure of how to move forward, but know that there are resources available to help you.

    Have you considered reaching out to a mental health professional? A therapist or counselor can provide you with support and guidance as you navigate your feelings. They can also help you develop coping strategies and explore treatment options.

    In the meantime, there are some self-care techniques that may help you feel better. These can include things like


In [20]:
!pip install datasets evaluate sacrebleu nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.7 MB/s eta 0:00:00


In [21]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=220d02073c97fedee8feea378f9a0ed02b85cedb464d5fbbece563306e3b2bb9
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [22]:
import evaluate
import math
import torch
import nltk
from transformers import AutoModelForCausalLM, AutoTokenizer

nltk.download("punkt")  # Needed for tokenization in BLEU & ROUGE

# Load BLEU and ROUGE metrics from Hugging Face's Evaluate library
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

# Function to calculate perplexity
def calculate_perplexity(response):
    inputs = tokenizer(response, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss
    perplexity = math.exp(loss.item())
    return perplexity


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [23]:
def evaluate_response(generated_response, reference_response):
    # Compute BLEU score (expects full sentences, not tokenized words)
    bleu_score = bleu.compute(predictions=[generated_response], references=[[reference_response]])

    # Compute ROUGE score (expects full sentences)
    rouge_score = rouge.compute(predictions=[generated_response], references=[reference_response])

    # Compute perplexity
    perplexity = calculate_perplexity(generated_response)

    return {
        "BLEU Score": bleu_score["bleu"],
        "ROUGE Score": rouge_score["rouge1"],
        "Perplexity": perplexity
    }


In [24]:
 import nltk
 nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [25]:
# Example chatbot response vs reference therapist response
generated_response = """
I'm sorry to hear that you're feeling anxious. It's completely normal to experience anxiety at times, but it's important to address it when it becomes overwhelming.
One helpful technique to manage anxiety is mindfulness meditation. This involves focusing on the present moment and accepting your thoughts and feelings without judgment.
"""
reference_response = """
I hear that you're feeling anxious, and that's completely understandable. A helpful way to manage anxiety is practicing deep breathing exercises or grounding techniques.
Taking a moment to focus on your breath or journaling can also be beneficial. Would you like to explore more coping strategies together?
"""

# Run evaluation
scores = evaluate_response(generated_response, reference_response)
print(scores)


{'BLEU Score': 0.12811300935224262, 'ROUGE Score': np.float64(0.38834951456310685), 'Perplexity': 2.6409495669289966}


In [26]:
!pip install -U sentence-transformers



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.4.1
    Uninstalling sentence-transformers-3.4.1:
      Successfully uninstalled sentence-transformers-3.4.1


In [27]:
!pip install faiss-cpu langchain chromadb


In [28]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load a free embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Free alternative to OpenAIEmbeddings

# Example dataset
conversation_texts = [
    "How do I deal with anxiety?",
    "What should I do if I'm feeling very anxious?",
    "How can I manage stress effectively?",
    "I've been feeling overwhelmed at work, how do I handle it?"
]

# Convert questions to vector embeddings
vectors = embedding_model.encode(conversation_texts, convert_to_numpy=True)

# Create FAISS index
vector_array = np.array(vectors, dtype=np.float32)
faiss_index = faiss.IndexFlatL2(vector_array.shape[1])
faiss_index.add(vector_array)

print("✅ FAISS Index Created Successfully with SentenceTransformers!")



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ FAISS Index Created Successfully with SentenceTransformers!


In [29]:
def retrieve_context(query):
    """Find similar past questions from FAISS and return relevant context."""
    query_vector = np.array([embedding_model.encode(query)], dtype=np.float32)
    distances, indices = faiss_index.search(query_vector, k=2)  # Retrieve top 2 similar questions

    # Fetch similar past questions
    similar_texts = [conversation_texts[i] for i in indices[0]]
    return similar_texts

# ✅ Test retrieval
query = "I'm feeling really stressed lately. What should I do?"
retrieved_context = retrieve_context(query)
print("🔍 Retrieved Context:", retrieved_context)


🔍 Retrieved Context: ['How can I manage stress effectively?', "What should I do if I'm feeling very anxious?"]


In [31]:
def generate_response_with_rag(client_message):
    # Retrieve similar past conversations
    context = retrieve_context(client_message)

    # Format prompt with retrieved knowledge
    prompt =  f"""
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Client: {client_message}
    Therapist:
    """

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=200, temperature=0.7, top_p=0.9)

    # Decode and return the response
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ✅ Test the chatbot with RAG
test_message = "hello iam sad"
response = generate_response_with_rag(test_message)
print("\nChatbot Response with RAG:\n", response)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Chatbot Response with RAG:
 
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Client: hello iam sad
    Therapist:
    
    Hello there. I'm sorry to hear that you're feeling sad. It's important to remember that it's okay to feel this way sometimes. Can you tell me more about what's been going on in your life that's been causing you to feel this way?


In [43]:
def generate_response_with_rag(client_message):
    # Retrieve similar past conversations
    context = retrieve_context(client_message)

    # Format prompt with retrieved knowledge
    prompt =  f"""
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Relevant Context: {context}

    Client: {client_message}
    Therapist:
    """

    # Tokenize input
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate response
    with torch.no_grad():
        outputs = model.generate(**inputs,max_new_tokens=200, temperature=0.7, top_p=0.9,eos_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Split at 'Therapist:' and return only the actual answer
    response = response.split("Therapist:")[-1].strip()
    return response


In [44]:
test_messages = [
    "hello i am sad",
    "i feel overwhelmed by everything at work",
    "my mom and wife keep arguing and it’s stressing me out",
    "i can't sleep at night and keep overthinking"
]

for msg in test_messages:
    print("\nClient:", msg)
    print("Therapist:", generate_response_with_rag(msg))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Client: hello i am sad


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Therapist: Hello, I'm sorry to hear that you're feeling sad. It's important to remember that it's okay to feel this way and that you're not alone. Can you tell me more about what's been going on and how you're feeling?

Client: i feel overwhelmed by everything at work


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Therapist: I'm sorry to hear that you're feeling overwhelmed at work. It sounds like things are becoming too much to handle. It's important to remember that it's okay to feel overwhelmed, and that there are ways to manage these feelings.

    One helpful technique is to practice mindfulness. This means being present in the moment and focusing on your thoughts and feelings without judgment. You can try taking a few deep breaths and focusing on your breath, or simply observing your thoughts and emotions without getting caught up in them.

    Another helpful strategy is to prioritize your tasks. Make a list of everything you need to do, and then rank them in order of importance. This can help you focus on the most pressing tasks first and prevent yourself from feeling overwhelmed.

    It's also important to take care of yourself outside of work. Make sure you're getting enough sleep, eating well, and engaging in regular physical activity. These self

Client: my mom and wife keep arguing

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Therapist: I'm sorry to hear that you're feeling stressed due to the arguments between your mom and wife. It's understandable that this can be a difficult situation to navigate, especially when it involves people you love.

    It's important to remember that everyone has their own unique perspective and way of communicating. It's possible that there may be misunderstandings or unspoken feelings that are contributing to the arguments.

    One approach you could try is to encourage both your mom and wife to sit down and have a conversation together. This can give them a chance to express their thoughts and feelings in a calm and respectful manner. It may also help them to better understand each other's perspectives and find common ground.

    If this doesn't work, it may be helpful to seek the support of a therapist who can help facilitate the conversation and provide guidance on effective communication skills.

    In the meantime, it'

Client: i can't sleep at night and keep overthi

In [32]:
# ✅ Test the chatbot with RAG
test_message = "I am not feeling good , what should i do?"
response = generate_response_with_rag(test_message)
print("\nChatbot Response with RAG:\n", response)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Chatbot Response with RAG:
 
    You are a compassionate therapist. Respond with empathy, support, and actionable advice.
    Maintain a warm and understanding tone. If necessary, suggest mindfulness, therapy, or self-care techniques.

    Client: I am not feeling good , what should i do?
    Therapist:
    
    I'm sorry to hear that you're not feeling well. It's important to take care of yourself and prioritize your well-being. Here are some suggestions that may help you feel better:

    1. Practice mindfulness: Mindfulness can help you stay present in the moment and reduce stress. You can try guided meditations, deep breathing exercises, or simply focusing on your breath.
    
    2. Engage in self-care: Taking care of yourself is essential for your well-being. You can try activities such as getting enough sleep, eating a healthy diet, exerc


In [45]:
!pip install transformers accelerate flask flask-ngrok

